https://adventofcode.com/2022/day/15

In [4]:
f = open("input.txt", "r")

s = read(f, String)

data = s

close(f)

# parses the coordinates of sensor and their beacons
function getData(str)
  lines =  str |> strip |> str -> split(str, "\n")
	coordinates = map(line -> map(coordinates -> parse(Int,coordinates.match),eachmatch(r"-?\d+",line)), lines)
  data = []
	for (x_sensor, y_sensor, x_beacon, y_beacon) in coordinates
		push!(data,[[x_sensor, y_sensor],[x_beacon, y_beacon]])
	end
	return data
end

getData (generic function with 1 method)

In [5]:
struct Beacon
	x
	y
	sensors
end

struct Sensor
	x
	y
	beacon
	manhatten_distance
end

# creates beacon and sensor objects base on the data
function generateObjects(data)
	beacon_list = []
	beacons = []
	sensors = []

	# loops through the data creating sensors and beacons based on the coordinate data
	for (sensor_coordinates, beacon_coordinates) in data[1:end]
		beacon_x, beacon_y = beacon_coordinates
		sensor_x, sensor_y = sensor_coordinates

		beacon = Beacon(beacon_x, beacon_y, [])

		# check in a beacon has already been created and retrieves it else create a new one
		if beacon_coordinates in beacon_list
			beacon_index = findfirst((beacon)-> beacon_x == beacon.x && beacon_y == beacon.y, beacons)
			beacon = beacons[beacon_index]
		else
			push!(beacon_list, beacon_coordinates)
			push!(beacons, beacon)
		end

		manhatten_distance = getManhattenDistance(sensor_x, sensor_y, beacon_x, beacon_y)
		sensor = Sensor(sensor_x, sensor_y, beacon, manhatten_distance)

		push!(beacon.sensors, sensor)
		push!(sensors, sensor)
	end

	return beacons, sensors
end

# calculates manhatten distances of coordinates
function getManhattenDistance(x₁, y₁, x₂, y₂)
	distance = abs(x₁-x₂) + abs(y₁-y₂)
end

# calculates the coordinates of intervals that are covered by sensors
function getRowCoverageIntervals(sensors, row)
	intervals = []

	for sensor in sensors
		# finds the midpoint of the row that the sensor and intersects
		intersect_x, intersect_y = [sensor.x, row]

		# calculates the distance from the midpoint to the sensor
		distance = getManhattenDistance(sensor.x, sensor.y, intersect_x, intersect_y)

		# if the midpoint is farther that the sensor's manhatten distance from it's beacon it's not exclude from having a beacon
		if distance <= sensor.manhatten_distance
			sensor_interval = getSensorInterval(sensor,distance,intersect_x, intersect_y)
			push!(intervals,sensor_interval)
		end
	end

	return intervals
end

# calculates the range of coordinates the beacon can't be in
function getSensorInterval(sensor,distance,intersect_x, intersect_y)
	# calculates the distance from the midpoint the sensor covers
	range_from_intersect = sensor.manhatten_distance - distance

	# gets coordinates from the left and right most interval
	left_x = intersect_x-range_from_intersect
	left_coordinates = [left_x, intersect_y]

	right_x = intersect_x+range_from_intersect
	right_coordinates = [right_x, intersect_y]

	return [left_coordinates, right_coordinates]
end

# merges the intervals if they can be merged
function mergeIntervals(intervals)
	sort!(intervals)
	result = []

	current = intervals[1]

	for next in intervals[2:end]
		if next[1][1] - current[2][1] <= 1
			left = min(current[1], next[1])
			right = max(current[2], next[2])
			current = [left, right]
		else
			push!(result,current)
			current = next
		end
	end

	push!(result,current)

	return result
end

# calculates the number of places the beacon can't be
function countCoverage(beacons, sensors, row)
	# gets the intervals the beacon can't be and merges them
	intervals = getRowCoverageIntervals(sensors, row)
	merged_intervals = mergeIntervals(intervals)

	count = 0

	# sums up the distance each interval covers
	for interval in merged_intervals
		count += interval[2][1] - interval[1][1] + 1
	end

	# removes the count of beacons in the row
	count -= getBeaconCountInRow(beacons, sensors, row)


	return count
end

# calculates how many beacons are a given row
function getBeaconCountInRow(beacons, sensors, row)
	count = 0

	for beacon in beacons
		if beacon.y == row
			count += 1
		end
	end 

	return count
end

# gets the count of exclude beacon positions
function getBeaconCoverageCount(data, row)
	data = getData(data)
	beacons, sensors = generateObjects(data)
	
	countCoverage(beacons, sensors, row)
end

getBeaconCoverageCount(data, 2000000)


4665948

In [6]:
function getDistressSignal(data)
	data = getData(data)
	beacons, sensors = generateObjects(data)

	# create a distress beacon object where the sensors don't cover
	distress_beacon = getDistressBeacon(sensors)

	signal = distress_beacon.x*4000000+distress_beacon.y

	return signal
end

function getDistressBeacon(sensors)
	distress_line_intervals = []

	# gets all the intervals for each row
	for i = 1:4000000
		intervals = getRowCoverageIntervals(sensors, i)
		merged_intervals = mergeIntervals(intervals)
		# if there is a gap in the intervals there will be more than one interval after merging
		if length(merged_intervals) > 1
			distress_line_intervals = merged_intervals
			break
		end
	end

	# the distress beacon will be x+1 from the end of the first interval
	distress_x = distress_line_intervals[1][2][1]+1
	distress_y = distress_line_intervals[1][2][2]

	beacon = Beacon(distress_x, distress_y, [])

	return beacon
end

getDistressSignal(data)

13543690671045